In [142]:
import random
import copy
import pandas as pd
import math
import statistics

max_days_in_row=6
start=1
num_days_in_month=31
allow_day_night_double=False #False
allow_night_day_double=False

prev_month={'A': {'Day1': [4, 5, 13, 18, 20, 25],
  'Day2': [8, 9, 14, 15, 19],
  'Night': [1, 2, 6, 21, 22, 26, 27, 28]},
 'B': {'Day1': [14, 15, 19, 21, 22, 27, 28, 30],
  'Day2': [3, 4, 7, 12, 13, 20, 26, 29, 31],
  'Night': [8, 9]},
 'C': {'Day1': [6, 17],
  'Day2': [1, 2, 16, 21, 22, 27, 28],
  'Night': [3, 7, 10, 11, 12, 13, 18, 23, 24, 29]},
 'D': {'Day1': [3, 8, 9, 11, 23, 24, 29],
  'Day2': [10],
  'Night': [4, 5, 14, 15, 16, 17, 20, 25, 30, 31]},
 'E': {'Day1': [1, 2, 7, 10, 12, 16, 26, 31],
  'Day2': [5, 6, 11, 17, 18, 23, 24, 25, 30],
  'Night': [19]}}

# prev_month={'A': {'Day1': [],
#   'Day2': [],
#   'Night': []},
#  'B': {'Day1': [],
#   'Day2': [],
#   'Night': []},
#  'C': {'Day1': [],
#   'Day2': [],
#   'Night': []},
#  'D': {'Day1': [],
#   'Day2': [],
#   'Night': []},
#  'E': {'Day1': [],
#   'Day2': [],
#   'Night': []}}

def modify_prev_month(prev_month):
    converted_prev_month=copy.deepcopy(prev_month)
    tot_prev_shifts={'A':[],'B':[],'C':[],'D':[],'E':[]}
    largest=0
    temp_largest=0
    for doc in converted_prev_month:
        for shift in converted_prev_month[doc]:
            if len(converted_prev_month[doc][shift])>0:
                temp_largest=max(converted_prev_month[doc][shift])
            if temp_largest>largest:
                largest=temp_largest
    for doc in converted_prev_month:
        num_shifts=0
        for shift in converted_prev_month[doc]:
            temp_array=[]
            for day in converted_prev_month[doc][shift]:
                if day>largest-6:
                    num_shifts=num_shifts+1
                    mapped_day=day-largest
                    temp_array.append(mapped_day)
            converted_prev_month[doc][shift]=temp_array
            tot_prev_shifts[doc]=num_shifts
    return converted_prev_month,tot_prev_shifts


def initialize_monthly_scheduler(num_days_in_month, doctor_shifts,prev_month=None):
    calendar = {}
    tot_prev_shifts={}
    for day in range(1, num_days_in_month + 1):
        day_shifts = ["Day1", "Day2", "Night"]
        calendar[day] = day_shifts
    scheduler = {}
    if prev_month==None:
        for doctor in doctor_shifts:
            scheduler[doctor] = {"Day1": [], "Day2": [], "Night": []}
    else:
        scheduler,tot_prev_shifts=modify_prev_month(prev_month)
    return calendar, scheduler, doctor_shifts, tot_prev_shifts

def has_3consecutive_shifts_single_doctor(doctor_schedule, proposed_day, proposed_shift_type):
    day_shifts = doctor_schedule["Day1"] + doctor_schedule["Day2"]
    night_shifts = doctor_schedule["Night"]
    if proposed_shift_type == "Day1":
        if proposed_day in night_shifts and (proposed_day + 1 in day_shifts):
            return True
        elif proposed_day-1 in night_shifts and proposed_day in night_shifts:
            return True
        elif proposed_day-1 in night_shifts and proposed_day-1 in day_shifts:
            return True 
    elif proposed_shift_type == "Day2":
        if proposed_day in night_shifts and proposed_day + 1 in day_shifts:
            return True
        elif proposed_day-1 in night_shifts and proposed_day in night_shifts:
            return True
        elif proposed_day-1 in night_shifts and proposed_day-1 in day_shifts:
            return True
    elif proposed_shift_type == "Night":
        if (proposed_day in day_shifts and proposed_day + 1 in day_shifts):
            return True
        if (proposed_day-1 in night_shifts and proposed_day  in day_shifts):
            return True 
        if (proposed_day in night_shifts and proposed_day+1  in day_shifts):
            return True 
        if (proposed_day+1 in night_shifts and proposed_day+1  in day_shifts):
            return True 
    return False

def is_schedule_legal(schedule, doctor_shifts, days_unavail, max_days_in_row,display=False,tot_prev_shifts=None):
    if tot_prev_shifts==None:
        tot_prev_shifts={}
        for doc in schedule:
            tot_prev_shifts[doc]=0       
    if has_consecutive_shifts(schedule):
        return False, f"Doctor scheduled for three consecutive shifts."
    if check_min_shift_types(schedule):
        return False, f"Doctor does not have correct number of shift_types."
    flattened=flatten_schedule(schedule)
    for flattened_sched in flattened.values():
        if check_max_consec_days(flattened_sched, num_days_in_month, max_days_in_row):
            return False,f"Doctor scheduled for too many straight days shifts."
    for doctor, shifts in schedule.items():
        # Check for two day shifts on the same day
        day_shifts = shifts['Day1'] + shifts['Day2']
        if len(day_shifts) != len(set(day_shifts)):
            return False, f"Doctor {doctor} is scheduled for two day shifts on the same day."
        
        #if day_night_double not allowed check if this exists
        if allow_day_night_double==False:
            if set(day_shifts).intersection(shifts['Night']):
                return False, f"Day Night Doubles not allowed."
        if allow_night_day_double==False:
            temp_days=[x - 1 for x in day_shifts]
            if set(temp_days).intersection(shifts['Night']):
                return False, f"Night Day Doubles not allowed."
        # Check if the total shifts for the doctor are within the min and max range
        total_shifts = len(day_shifts) + len(shifts['Night'])
        min_shifts, max_shifts = doctor_shifts[doctor]
        if (total_shifts-tot_prev_shifts[doctor]) < min_shifts or (total_shifts-tot_prev_shifts[doctor]) > max_shifts:
            return False, f"Doctor {doctor} has an invalid number of shifts."
            
        # Check if the doctor is scheduled for shifts they are unavailable on
        for shift_type, days in shifts.items():
            #print(shift_type," ",days)
            for day in days:
                if shift_type.startswith("Day"):
                    shft_tmp="Day"
                else:
                    shft_tmp="Night"
                if day_type := days_unavail[doctor][shft_tmp]:
                    if day in day_type:
                        return False, f"Doctor {doctor} is scheduled for an unavailable shift on day {day}."
    if display==True:
        if allow_day_night_double==False:
            print("No day to night double shifts scheduled")
        if allow_night_day_double==False:
            print("No night to day double shifts scheduled")
        print("No one scheduled for more than ",max_days_in_row," in a row")
        print("No one scheduled on their unavailable day")
        print("all shift number constraints are complied with")
    return True, "Schedule is legal."

def flatten_schedule(schedule):   #used to calculate schedule pattern loss
    flattened_schedule = {}
    for doctor, shifts in schedule.items():
        work_days = []
        for shift_type, days in shifts.items():
            if shift_type.startswith("Day") or shift_type == "Night":
                work_days.extend(days)
        work_days.sort()
        flattened_schedule[doctor] = work_days
    return flattened_schedule

def req_daysoff_score(schedule, req_days_off):
    conflict_count = 0
    for doctor, shifts in schedule.items():
        day_shifts=shifts["Day1"]+shifts["Day2"]
        day_conflicts=set(day_shifts).intersection(req_days_off[doctor]['Day'])
        night_conflicts=set(shifts['Night']).intersection(req_days_off[doctor]['Night'])
        conflict_count=conflict_count+len(day_conflicts)+len(night_conflicts)
    return conflict_count

def doctor_schedule_pattern_score(numbers,num_days_in_month):  #takes in value of the flatten_schedule dictionary output individually
    consecutive_count = 0
    gap_count = 0
    consecutive_counts = []
    gap_counts = []
    start_consec_indx=[]
    start=min(numbers)
    last_number = start - 1
    start_consec_indx.append(start)
    for number in numbers:
        if number - last_number <= 1:
            consecutive_count += 1
        else:
            start_consec_indx.append(number)
            if consecutive_count > 0:
                consecutive_counts.append(consecutive_count)

            consecutive_count = 1

            gap_count = number - last_number - 1
            if gap_count > 0:
                gap_counts.append(gap_count)
        
        last_number = number

    consecutive_counts.append(consecutive_count)

    # Add a final gap count if the end number is greater than the biggest number in the list, considering it inclusive
    final_gap = num_days_in_month - numbers[-1]
    if final_gap > 0:
        gap_counts.append(final_gap)

    target_work, target_off=4.5, 2.5
    work_days_score=0
    i=0
    for number in consecutive_counts:
        if number==1 and start_consec_indx[i]!=num_days_in_month:
            work_days_score=work_days_score+7
        elif number==2 and start_consec_indx[i]!=num_days_in_month-1:
            work_days_score=work_days_score+5
        elif number==3 and start_consec_indx[i]!=num_days_in_month-2:
            work_days_score=work_days_score+abs(target_work - number)
        elif number==4 and start_consec_indx[i]!=num_days_in_month-3:
            work_days_score=work_days_score+abs(target_work - number)
        elif number==5 and start_consec_indx[i]!=num_days_in_month-4:
            work_days_score=work_days_score+abs(target_work - number)
        elif number>5:
            work_days_score=work_days_score+abs(target_work - number)
        i=i+1
        
    off_days_score=0
    for number in gap_counts:
        if number==1:
            off_days_score=off_days_score+2

    return work_days_score,off_days_score

def num_d2_shift_score(schedule):
    d2s=[]
    for doc in schedule:
        d2s.append(len(schedule[doc]['Day2'])-num_shifts_prev_month(schedule[doc]['Day2']))
    var=statistics.variance(d2s)
    var=var/.5
    return var


def schedule_pattern_loss(num_days_in_month,doctor_schedules):  #uses above two functions
    flattened=flatten_schedule(doctor_schedules)
    loss=0
    for value in flattened.values():
        on,off=doctor_schedule_pattern_score(value, num_days_in_month)
        loss=loss+on+1.8*off
    return loss


def count_total_shifts(doctor_schedule):
    total_shifts = sum(len(shifts) for shifts in doctor_schedule.values())
    return total_shifts

def doctors_below_minimum(doctor_shifts, scheduler, tot_prev_shifts):
    below_minimum = []
    for doctor, shifts_range in doctor_shifts.items():
        min_shifts, _ = shifts_range
        total_shifts = sum(len(scheduler[doctor][shift_type]) for shift_type in scheduler[doctor])-tot_prev_shifts[doctor]
        if total_shifts < min_shifts:
            below_minimum.append(doctor)
    return below_minimum

 
def has_consecutive_shifts(schedule):
    for doctor, shifts in schedule.items():
        for day in shifts['Day1']:
            if day in shifts['Night']:
                if (day+1) in shifts['Day1'] or (day+1) in shifts['Day2']:
                    return True
                if (day-1) in shifts['Night']:
                    return True
        for day in shifts['Day2']:
            if day in shifts['Night']:
                if (day+1) in shifts['Day1'] or (day+1) in shifts['Day2']:
                    return True
                if (day-1) in shifts['Night']:
                    return True
        for day in shifts['Night']:
            if (day+1) in shifts['Day1'] or (day+1) in shifts['Day2']:
                if (day+1) in shifts['Night']:
                    return True
            if day in shifts['Day1'] or day in shifts['Day2']:
                if (day-1) in shifts['Night']:
                    return True
    return False

def modify_schedule(schedule, month_days):  #proposed updated schedule (changes single day)
    # Step 1: Randomly select a shift and a date
    shifts = ['Day1', 'Day2', 'Night']
    shift = random.choice(shifts)
    date =random.randint(1, month_days)

    # Step 2: Randomly select a doctor
    doctors = list(schedule.keys())
    chosen_doctor = random.choice(doctors)
    #print(shift," ",date," ",chosen_doctor)
    # Step 3: Remove the selected date from the doctor who originally had it
    for doctor in doctors:
        if date in schedule[doctor][shift]:
            schedule[doctor][shift].remove(date)
            break

    # Step 4: Add the selected date to the same shift for the randomly chosen doctor
    schedule[chosen_doctor][shift].append(date)
    schedule[chosen_doctor][shift].sort()  # Optional: Sort for better readability
    return schedule

def check_max_consec_days(numbers, num_days_in_month, max_days):  #takes in value of the flatten_schedule dictionary output individually
    start=min(numbers)
    consecutive_count = 0
    gap_count = 0
    consecutive_counts = []
    gap_counts = []

    last_number = start - 1

    for number in numbers:
        if number - last_number <= 1:
            consecutive_count += 1
        else:
            if consecutive_count > 0:
                consecutive_counts.append(consecutive_count)
            consecutive_count = 1

            gap_count = number - last_number - 1
            if gap_count > 0:
                gap_counts.append(gap_count)

        last_number = number
        if consecutive_count>max_days:
            return True
    return False    

def shift_variation_score(doctor_schedules,ideal_num_shifts,tot_prev_shifts):
    variation_from_ideal_num_shifts=0
    for each in ideal_num_shifts:
        variation_from_ideal_num_shifts=variation_from_ideal_num_shifts+abs(ideal_num_shifts[each]-(count_total_shifts(doctor_schedules[each])-tot_prev_shifts[each]))
    return variation_from_ideal_num_shifts

def shifttype_pref_score(doctor_schedules,shift_type_pref):
    shift_pref_score=0
    for each in doctor_schedules:
        if shift_type_pref[each]=="Night":
            shift_pref_score_temp=(len(doctor_schedules[each]['Day1'])+len(doctor_schedules[each]['Day2']))/count_total_shifts(doctor_schedules[each])
        else:
            shift_pref_score_temp=len(doctor_schedules[each]['Night'])/count_total_shifts(doctor_schedules[each])
        shift_pref_score=shift_pref_score+shift_pref_score_temp
    return shift_pref_score           

def assign_shifts(calendar, scheduler, doctor_shifts, days_unavail,tot_prev_shifts):
    remaining_shifts = [(day, shift) for day in calendar for shift in calendar[day]]
    
    while remaining_shifts:
        day, shift_type = random.choice(remaining_shifts)
        docs_below = doctors_below_minimum(doctor_shifts, scheduler,tot_prev_shifts)
        eligible_doctors = []
        for doctor in docs_below:
            too_many_straight_shifts=False
            flattened=flatten_schedule(scheduler)
            flattened_doc=flattened[doctor]
            temp=copy.deepcopy(flattened_doc)
            temp.append(day)
            temp.sort()             
            if check_max_consec_days(temp, num_days_in_month, max_days_in_row):
                    too_many_straight_shifts=True
            
            if too_many_straight_shifts==True:
                continue
            elif (count_total_shifts(scheduler[doctor])-tot_prev_shifts[doctor]) > doctor_shifts[doctor][1]:
                continue
            elif  has_3consecutive_shifts_single_doctor(scheduler[doctor], day, shift_type):
                continue
            elif shift_type.startswith("Day"):
                if day in scheduler[doctor]["Day1"] or day in scheduler[doctor]["Day2"]:
                    continue
                elif day in days_unavail[doctor]["Day"] and shift_type.startswith("Day"):  # Check if the day and shift are unavailable for the doctor
                    continue
                elif day in days_unavail[doctor]["Night"] and shift_type=="Night":  # Check if the day and shift are unavailable for the doctor
                    continue
                elif allow_day_night_double==False and day in scheduler[doctor]['Night']:
                    continue
                elif allow_night_day_double==False and (day-1) in scheduler[doctor]['Night']:
                        continue
                else:
                    eligible_doctors.append(doctor)
            else:
                if day in scheduler[doctor][shift_type]:
                    continue
                elif day in days_unavail[doctor][shift_type]:  # Check if the day and shift are unavailable for the doctor
                    continue
                elif allow_day_night_double==False and (day in scheduler[doctor]["Day1"] or day in scheduler[doctor]["Day2"]):
                    continue   #don't allow a night followed by day double
                elif allow_night_day_double==False and (day+1 in scheduler[doctor]["Day1"] or day+1 in scheduler[doctor]["Day2"]):
                    continue   #don't allow a night followed by day double
                else:
                    eligible_doctors.append(doctor)
        
        if eligible_doctors == []:
            for doctor in doctor_shifts:
                too_many_straight_shifts=False
                flattened=flatten_schedule(scheduler)
                flattened_doc=flattened[doctor]
                temp=copy.deepcopy(flattened_doc)
                temp.append(day)
                temp.sort()             
                if check_max_consec_days(temp, num_days_in_month, max_days_in_row):
                        too_many_straight_shifts=True
                        
                if too_many_straight_shifts==True:
                    continue                        
                elif (count_total_shifts(scheduler[doctor])-tot_prev_shifts[doctor]) >= doctor_shifts[doctor][1]:
                    continue
                elif  has_3consecutive_shifts_single_doctor(scheduler[doctor], day, shift_type):
                    continue
                elif shift_type.startswith("Day"):
                    if day in scheduler[doctor]["Day1"] or day in scheduler[doctor]["Day2"]:
                        continue
                    elif day in days_unavail[doctor]["Day"] and shift_type.startswith("Day"):  # Check if the day and shift are unavailable for the doctor
                        continue
                    elif day in days_unavail[doctor]["Night"] and shift_type=="Night":  # Check if the day and shift are unavailable for the doctor
                        continue
                    elif allow_day_night_double==False and day in scheduler[doctor]['Night']:
                        continue
                    elif allow_night_day_double==False and (day-1) in scheduler[doctor]['Night']:
                        continue
                    else:
                        eligible_doctors.append(doctor)
                else:
                    if day in scheduler[doctor][shift_type]:
                        continue
                    elif day in days_unavail[doctor][shift_type]:  # Check if the day and shift are unavailable for the doctor
                        continue
                    elif allow_day_night_double==False and (day in scheduler[doctor]["Day1"] or day in scheduler[doctor]["Day2"]):
                        continue   #don't allow a night followed by day double
                    elif allow_night_day_double==False and (day+1 in scheduler[doctor]["Day1"] or day+1 in scheduler[doctor]["Day2"]):
                        continue   #don't allow a night followed by day double
                    else:
                        eligible_doctors.append(doctor)

            if eligible_doctors == []:
                print("Cannot create schedule.")
                return
            
        doc=fewest_shifts(scheduler,shift_type, eligible_doctors)    
        scheduler[doc][shift_type].append(day) 
        remaining_shifts.remove((day, shift_type))
    
    print("Schedule created successfully.")

def print_schedule(calendar, scheduler):
    shift_types = ["Day1", "Day2", "Night"]
    
    # Print the header row with shift types
    print("   ", end="")
    for shift_type in shift_types:
        print(f"{shift_type} ", end="")
    print()
    
    for day, shifts in calendar.items():
        print(f"{day:2} ", end="")
        for shift_type in shifts:
            assigned_doctor = ""
            for doctor, doctor_shifts in scheduler.items():
                if day in doctor_shifts[shift_type]:
                    assigned_doctor = doctor
                    break
            print(f"{assigned_doctor:4}", end=" ")
        print()
        
def print_horizontal(doctor_schedules, num_days_in_month):
    # Create a DataFrame with the required structure
    df = pd.DataFrame(index=doctor_schedules.keys(), columns=range(1, num_days_in_month + 1))

    # Fill the DataFrame based on the schedule
    for doctor, shifts in doctor_schedules.items():
        for day in range(1, num_days_in_month + 1):
            shift_types = []
            if day in shifts.get('Day1', []):
                shift_types.append('D1')
            if day in shifts.get('Day2', []):
                shift_types.append('D2')
            if day in shifts.get('Night', []):
                shift_types.append('N')

            df.at[doctor, day] = 'DB' if len(shift_types) > 1 else ''.join(shift_types) or '·'

    # Styling function to apply colors
    def color_schedule(val):
        if val == 'D1':
            color = 'green'
        elif val == 'D2':
            color = 'blue'
        elif val == 'N':
            color = 'red'
        elif val == 'DB':
            color = 'purple'
        else:
            color = 'black'
        return f'color: {color}'

    return df.style.applymap(color_schedule)

##new
def num_shifts_prev_month(shift_dates):
    num_shifts = sum(1 for element in shift_dates if element < 1)
    return num_shifts

def fewest_shifts(schedule,shift,eligible_doctors):
    fewest=100
    for doc in eligible_doctors:
        num_shifts=len(schedule[doc][shift])-num_shifts_prev_month(schedule[doc][shift])
        if num_shifts<fewest:
            return_doc=doc
            fewest=num_shifts
    return return_doc      

def check_min_shift_types(schedule):
    for doc in schedule:
        for shift in schedule[doc]:
            num_shifts=len(schedule[doc][shift])-num_shifts_prev_month(schedule[doc][shift])
            if num_shifts<doctor_shift_types[doc][shift][0] or num_shifts>doctor_shift_types[doc][shift][1] :
                return True
    return False

##

###stats
def calculate_percentage_days_off(schedule, req_days_off):
    percentages = {}
    for doctor in schedule:
        # Initialize counts
        total_requested_days = 0
        days_off_matched = 0

        # Count total requested days and matched days off
        for shift_type in schedule[doctor]:
            if shift_type in ['Day1', 'Day2']:  # Day shifts
                day_type = 'Day'
            else:  # Night shifts
                day_type = 'Night'

            requested_days = req_days_off[doctor][day_type]
            total_requested_days += len(requested_days)
            days_off_matched += len([day for day in requested_days if day not in schedule[doctor][shift_type]])

        # Calculate percentage
        if total_requested_days > 0:
            percentage_off = (days_off_matched / total_requested_days) * 100
        else:
            percentage_off = 0  # Avoid division by zero

        percentages[doctor] = round(percentage_off, 2)  # Rounded to 2 decimal places
    for each in percentages:
        print(each," percent or requested days off actually off:",percentages[each])

def actual_vs_requested_shifts(schedule,ideal_num_shifts,tot_prev_shifts):
    for each in schedule:
        d1=len(schedule[each]['Day1'])-num_shifts_prev_month(schedule[each]['Day1'])
        d2=len(schedule[each]['Day2'])-num_shifts_prev_month(schedule[each]['Day2'])
        night=len(schedule[each]['Night'])-num_shifts_prev_month(schedule[each]['Night'])
        print(each," req:",ideal_num_shifts[each]," actual shifts:",count_total_shifts(doctor_schedules[each])-tot_prev_shifts[each], "Day1: ",d1," Day2: ",d2," Night: ",night)

def percentage_of_preferred_shift(schedule,shift_type_pref):
    for each in schedule:
        night_count=len([x for x in schedule[each]['Night'] if x > 0])
        d1_count=len([x for x in schedule[each]['Day1'] if x > 0])
        d2_count=len([x for x in schedule[each]['Day2'] if x > 0])

        if shift_type_pref[each]=="Night":
            per=night_count/(night_count+d1_count+d2_count)
        else:
            per=(d1_count+d2_count)/(night_count+d1_count+d2_count)

        print(each," percent of preferred shifts to actual shifts: ",per)
                                                                                             

num_days_in_month = 31  # Adjust as needed
doctor_shifts = {"A": [5, 21], "B": [5, 21], "C": [5, 21], "D": [5, 21], "E": [5, 21]}
doctor_shift_types={"A": {'Day1':[5,20], 'Day2':[0,20],'Night':[5,20]},"B": {'Day1':[5,20], 'Day2':[0,20],'Night':[5,20]},"C": {'Day1':[5,20], 'Day2':[0,20],'Night':[5,20]},"D": {'Day1':[5,20], 'Day2':[0,20],'Night':[5,20]},"E": {'Day1':[5,20], 'Day2':[0,20],'Night':[5,20]}}
ideal_num_shifts = {"A": 18, "B": 18, "C": 18, "D": 18, "E": 18}
#days_unavail = {"A": {"Day": [], "Night": []}, "B": {"Day": [], "Night": []}, "C": {"Day":[],"Night":[]}, "D": {"Day":[],"Night":[]}, "E": {"Day": [30], "Night": [30]}}
days_unavail = {"A": {"Day": [], "Night": []}, "B": {"Day": [], "Night": []}, "C": {"Day":[15],"Night":[]}, "D": {"Day":[],"Night":[15]}, "E": {"Day": [15], "Night": [15]}}

req_days_off = {"A": {"Day": [1,2,3], "Night": [1,2,3]}, "B": {"Day": [4,5,6], "Night": [4,5,6]}, "C": {"Day":[7,8,9],"Night":[7,8,9]}, "D": {"Day":[10,11,12],"Night":[10,11,12]}, "E": {"Day": [13,14,15], "Night": [13,14,15]}}
shift_type_pref={"A": "Day", "B": "Day", "C": "Day", "D": "Night", "E": "Night"}
monthly_calendar, doctor_schedules, doctor_shifts, tot_prev_shifts = initialize_monthly_scheduler(num_days_in_month, doctor_shifts,prev_month)
assign_shifts(monthly_calendar, doctor_schedules, doctor_shifts, days_unavail,tot_prev_shifts)
#print_schedule(monthly_calendar, doctor_schedules)
df=print_horizontal(doctor_schedules,num_days_in_month)
is_schedule_legal(doctor_schedules, doctor_shifts, days_unavail,max_days_in_row,True,tot_prev_shifts)
df

Schedule created successfully.
No day to night double shifts scheduled
No night to day double shifts scheduled
No one scheduled for more than  6  in a row
No one scheduled on their unavailable day
all shift number constraints are complied with


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A,D2,N,·,D1,D1,D2,·,D1,D2,N,·,D1,D2,·,D2,·,D1,·,·,N,·,N,N,N,·,·,D2,N,·,D2,D1
B,·,D2,D1,N,·,·,D1,D2,N,·,·,N,·,D2,N,N,·,·,D2,·,D1,D1,D2,·,D2,D1,·,D1,D1,N,N
C,·,·,·,D2,·,D1,N,N,·,·,D2,D2,N,N,·,D1,D2,D2,·,D1,N,·,D1,D1,·,N,·,D2,·,D1,·
D,N,·,N,·,N,N,·,·,·,D2,N,·,·,D1,D1,·,·,D1,D1,D2,·,D2,·,·,D1,D2,D1,·,D2,·,D2
E,D1,D1,D2,·,D2,·,D2,·,D1,D1,D1,·,D1,·,·,D2,N,N,N,·,D2,·,·,D2,N,·,N,·,N,·,·


In [66]:
#cur_loss = schedule_pattern_loss(1, 31, doctor_schedules)
#cur_loss=req_daysoff_score(doctor_schedules, req_days_off)
#cur_loss=shift_variation_score(doctors_schedules,ideal_num_shifts)
#cur_loss=shifttype_pref_score(doctor_schedules,shift_type_pref)
weight=[.1,.1,.7,.1]
pattern_loss = schedule_pattern_loss(31, doctor_schedules)
req_daysoff_loss=req_daysoff_score(doctor_schedules, req_days_off)
num_shift_var_loss=shift_variation_score(doctor_schedules,ideal_num_shifts,tot_prev_shifts)
prefer_shift_loss=shifttype_pref_score(doctor_schedules,shift_type_pref)
cur_loss=weight[0]*(pattern_loss-35)/(235-35)+weight[1]*req_daysoff_loss/25+weight[2]*num_shift_var_loss/25+weight[3]*prefer_shift_loss/3.5
print(cur_loss)

# Total iterations and initial modifications
total_iterations = 100000
initial_modifications = 10

for i in range(total_iterations):
    doctor_schedules_copy = copy.deepcopy(doctor_schedules)

    # Calculate the current number of modifications (simulated annealing effect)
    modifications = initial_modifications - (initial_modifications - 1) * i // total_iterations
    for j in range(modifications):
        sched_update = modify_schedule(doctor_schedules_copy, 31)
    #print(is_schedule_legal(sched_update, doctor_shifts, days_unavail, max_days_in_row,False,tot_prev_shifts)[0])
    if is_schedule_legal(sched_update, doctor_shifts, days_unavail, max_days_in_row,False,tot_prev_shifts)[0]:
        #print("here")
        pattern_loss = schedule_pattern_loss(31, sched_update)
        req_daysoff_loss=req_daysoff_score(sched_update, req_days_off)
        num_shift_var_loss=shift_variation_score(sched_update,ideal_num_shifts,tot_prev_shifts)
        prefer_shift_loss=shifttype_pref_score(sched_update,shift_type_pref)
        new_loss=weight[0]*(pattern_loss-35)/(235-35)+weight[0]*req_daysoff_loss/25+weight[0]*num_shift_var_loss/25+weight[0]*prefer_shift_loss/3.5
        if new_loss < cur_loss:
            doctor_schedules = sched_update
            cur_loss = new_loss
    
print(cur_loss)

0.3119753381194375
0.11538631120432363


In [91]:
#cur_loss = schedule_pattern_loss(1, 31, doctor_schedules)
#cur_loss=req_daysoff_score(doctor_schedules, req_days_off)
#cur_loss=shift_variation_score(doctors_schedules,ideal_num_shifts)
#cur_loss=shifttype_pref_score(doctor_schedules,shift_type_pref)
weight=[.25,.25,.25,.25]
pattern_loss = schedule_pattern_loss(31, doctor_schedules)
req_daysoff_loss=req_daysoff_score(doctor_schedules, req_days_off)
num_shift_var_loss=shift_variation_score(doctor_schedules,ideal_num_shifts,tot_prev_shifts)
prefer_shift_loss=shifttype_pref_score(doctor_schedules,shift_type_pref)
d2_shift_score=num_d2_shift_score(doctor_schedules)

cur_loss=d2_shift_score+weight[0]*(pattern_loss-35)/(235-35)+weight[1]*req_daysoff_loss/25+weight[2]*num_shift_var_loss/25+weight[3]*prefer_shift_loss/3.5
print(cur_loss)

# Total iterations and initial modifications
total_iterations = 100000
initial_modifications = 10

temperature=90
min_temperature=10
cooling_rate=.99

for i in range(total_iterations):
    doctor_schedules_copy = copy.deepcopy(doctor_schedules)

    # Calculate the current number of modifications (simulated annealing effect)
    modifications = initial_modifications - (initial_modifications - 1) * i // total_iterations
    for j in range(modifications):
        sched_update = modify_schedule(doctor_schedules_copy, 31)
    #print(is_schedule_legal(sched_update, doctor_shifts, days_unavail, max_days_in_row,False,tot_prev_shifts)[0])
    if is_schedule_legal(sched_update, doctor_shifts, days_unavail, max_days_in_row,False,tot_prev_shifts)[0]:
        #print("here")
        pattern_loss = schedule_pattern_loss(31, sched_update)
        req_daysoff_loss=req_daysoff_score(sched_update, req_days_off)
        num_shift_var_loss=shift_variation_score(sched_update,ideal_num_shifts,tot_prev_shifts)
        prefer_shift_loss=shifttype_pref_score(sched_update,shift_type_pref)
        new_loss=weight[0]*(pattern_loss-35)/(235-35)+weight[0]*req_daysoff_loss/25+weight[0]*num_shift_var_loss/25+weight[0]*prefer_shift_loss/3.5
                # Calculate acceptance probability
        if new_loss < cur_loss:
            accept = True
        else:
            delta = new_loss - cur_loss
            probability = math.exp(-delta / temperature)
            accept = random.random() < probability

        # Accept the new solution based on the acceptance probability
        if accept:
            doctor_schedules = sched_update
            cur_loss = new_loss

    # Decrease the temperature
    temperature *= cooling_rate  # cooling_rate < 1, for example, 0.99

    
print(cur_loss)

1.1658396900718329
0.2800957960259203


In [92]:
print_horizontal(doctor_schedules,num_days_in_month)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A,D1,D2,N,·,·,D2,D2,D1,D2,·,D1,D1,N,·,D2,N,N,N,·,·,D2,N,·,·,·,D1,D1,D2,D2,·,·
B,·,·,·,·,·,·,D1,D2,N,N,·,D2,D2,D2,D1,D2,·,·,D1,D2,N,·,D1,D1,D2,D2,·,·,·,N,N
C,D2,D1,D1,D1,D1,·,·,·,·,D1,N,N,·,D1,N,·,·,·,·,·,D1,D2,N,N,·,·,D2,D1,D1,D1,D2
D,·,·,D2,N,N,N,N,N,·,·,·,·,D1,·,·,D1,D1,D2,N,N,·,D1,D2,D2,D1,·,·,N,N,·,·
E,N,N,·,D2,D2,D1,·,·,D1,D2,D2,·,·,N,·,·,D2,D1,D2,D1,·,·,·,·,N,N,N,·,·,D2,D1


In [908]:
doctor_schedules
#print_horizontal(doctor_schedules,num_days_in_month)

{'A': {'Day1': [11, 12, 17, 19, 20, 26, 27],
  'Day2': [4, 5, 6, 7, 8, 18, 25, 28],
  'Night': [-5, -4, -3, 13, 14, 29, 30]},
 'B': {'Day1': [-4, -3, -1, 8, 9, 14, 15, 16, 23, 25, 29, 30, 31],
  'Day2': [-5, -2, 0, 2, 3, 13, 21, 22, 24],
  'Night': [10, 17]},
 'C': {'Day1': [10, 21, 22, 28],
  'Day2': [-4, -3, 1, 15, 16, 17, 23, 29, 30, 31],
  'Night': [-2, 2, 3, 4, 11, 12, 18, 24]},
 'D': {'Day1': [-2, 4, 5, 6, 7, 13],
  'Day2': [14, 19, 20, 26, 27],
  'Night': [-1, 0, 1, 8, 9, 15, 16, 21, 22, 23, 28]},
 'E': {'Day1': [-5, 0, 1, 2, 3, 18, 24],
  'Day2': [-1, 9, 10, 11, 12],
  'Night': [5, 6, 7, 19, 20, 25, 26, 27, 31]}}

In [93]:
calculate_percentage_days_off(doctor_schedules, req_days_off)
print(" ")
actual_vs_requested_shifts(doctor_schedules,ideal_num_shifts,tot_prev_shifts)	
print("")
percentage_of_preferred_shift(doctor_schedules,shift_type_pref)

A  percent or requested days off actually off: 66.67
B  percent or requested days off actually off: 100.0
C  percent or requested days off actually off: 100.0
D  percent or requested days off actually off: 100.0
E  percent or requested days off actually off: 88.89
 
A  req: 18  actual shifts: 20 Day1:  6  Day2:  8  Night:  6
B  req: 18  actual shifts: 18 Day1:  5  Day2:  8  Night:  5
C  req: 18  actual shifts: 19 Day1:  10  Day2:  4  Night:  5
D  req: 18  actual shifts: 18 Day1:  5  Day2:  4  Night:  9
E  req: 18  actual shifts: 18 Day1:  5  Day2:  7  Night:  6

A  percent of preferred shifts to actual shifts:  0.7
B  percent of preferred shifts to actual shifts:  0.7222222222222222
C  percent of preferred shifts to actual shifts:  0.7368421052631579
D  percent of preferred shifts to actual shifts:  0.5
E  percent of preferred shifts to actual shifts:  0.3333333333333333


In [895]:
calculate_percentage_days_off(doctor_schedules, req_days_off)
print(" ")
actual_vs_requested_shifts(doctor_schedules,ideal_num_shifts,tot_prev_shifts)	
print("")
percentage_of_preferred_shift(doctor_schedules,shift_type_pref)

A  percent or requested days off actually off: 77.78
B  percent or requested days off actually off: 100.0
C  percent or requested days off actually off: 100.0
D  percent or requested days off actually off: 88.89
E  percent or requested days off actually off: 88.89
 
A  req: 18  actual shifts: 20
B  req: 18  actual shifts: 18
C  req: 18  actual shifts: 19
D  req: 18  actual shifts: 18
E  req: 18  actual shifts: 18

A  percent of preferred shifts to actual shifts:  1.0
B  percent of preferred shifts to actual shifts:  0.8333333333333334
C  percent of preferred shifts to actual shifts:  0.7368421052631579
D  percent of preferred shifts to actual shifts:  0.8888888888888888
E  percent of preferred shifts to actual shifts:  0.3888888888888889


In [782]:
req_days_off = {"A": {"Day": [1,2,3], "Night": [1,2,3]}, "B": {"Day": [4,5,6], "Night": [4,5,6]}, "C": {"Day":[7,8,9],"Night":[7,8,9]}, "D": {"Day":[10,11,12],"Night":[10,11,12]}, "E": {"Day": [13,14,15], "Night": [13,14,15]}}
doctor_schedules

{'A': {'Day1': [4, 15, 16, 21, 26, 27, 29],
  'Day2': [8, 9, 10, 11, 14, 17, 20, 28],
  'Night': [-5, -4, -3, 5, 6, 22, 23]},
 'B': {'Day1': [-4, -3, -1, 3, 7, 8, 9, 10, 17, 19, 31],
  'Day2': [-5, -2, 0, 2, 15, 16, 18, 26, 30],
  'Night': [11, 12]},
 'C': {'Day1': [1, 2, 11, 14],
  'Day2': [-4, -3, 3, 4, 5, 6, 12, 13, 21, 22, 23, 27],
  'Night': [-2, 15, 24, 25, 28, 29, 30]},
 'D': {'Day1': [-2, 5, 6, 18, 20, 24, 25, 30],
  'Day2': [19],
  'Night': [-1, 0, 1, 7, 8, 9, 13, 21, 26, 27, 31]},
 'E': {'Day1': [-5, 0, 12, 13, 22, 23, 28],
  'Day2': [-1, 1, 7, 24, 25, 29, 31],
  'Night': [2, 3, 4, 10, 14, 16, 17, 18, 19, 20]}}

In [907]:
is_schedule_legal(doctor_schedules, doctor_shifts, days_unavail, max_days_in_row,display=True,tot_prev_shifts=tot_prev_shifts)

No day to night double shifts scheduled
No night to day double shifts scheduled
No one scheduled for more than  6  in a row
No one scheduled on their unavailable day
all shift number constraints are complied with


(True, 'Schedule is legal.')

In [829]:
def schedule_pattern_loss(num_days_in_month,doctor_schedules):  #uses above two functions
    flattened=flatten_schedule(doctor_schedules)
    loss=0
    for value in flattened.values():
        on,off=doctor_schedule_pattern_score(value, num_days_in_month)
        loss=loss+on+1.8*off
    return loss

In [822]:
flattened=flatten_schedule(doctor_schedules)

In [826]:
a=flattened['A']
print(a)

[-5, -4, -3, 4, 7, 8, 9, 10, 11, 14, 15, 16, 17, 21, 22, 23, 26, 27, 28, 29, 30, 31]


In [832]:
tst=[1,2,3,4,8,9,10,11,15,16,17,18,22,23,24,25,26,30,31]
len(tst)

19

In [862]:
doctor_schedule_pattern_score(tst,31)

(2.0, 0)

In [861]:
def doctor_schedule_pattern_score(numbers,num_days_in_month):  #takes in value of the flatten_schedule dictionary output individually
    consecutive_count = 0
    gap_count = 0
    consecutive_counts = []
    gap_counts = []
    start_consec_indx=[]
    start=min(numbers)
    last_number = start - 1
    start_consec_indx.append(start)
    for number in numbers:
        if number - last_number <= 1:
            consecutive_count += 1
        else:
            start_consec_indx.append(number)
            if consecutive_count > 0:
                consecutive_counts.append(consecutive_count)

            consecutive_count = 1

            gap_count = number - last_number - 1
            if gap_count > 0:
                gap_counts.append(gap_count)
        
        last_number = number

    consecutive_counts.append(consecutive_count)

    # Add a final gap count if the end number is greater than the biggest number in the list, considering it inclusive
    final_gap = num_days_in_month - numbers[-1]
    if final_gap > 0:
        gap_counts.append(final_gap)

    target_work, target_off=4.5, 2.5
    work_days_score=0
    i=0
    for number in consecutive_counts:
        if number==1 and start_consec_indx[i]!=num_days_in_month:
            work_days_score=work_days_score+7
        elif number==2 and start_consec_indx[i]!=num_days_in_month-1:
            work_days_score=work_days_score+5
        elif number==3 and start_consec_indx[i]!=num_days_in_month-2:
            work_days_score=work_days_score+abs(target_work - number)
        elif number==4 and start_consec_indx[i]!=num_days_in_month-3:
            work_days_score=work_days_score+abs(target_work - number)
        elif number==5 and start_consec_indx[i]!=num_days_in_month-4:
            work_days_score=work_days_score+abs(target_work - number)
        elif number>5:
            work_days_score=work_days_score+abs(target_work - number)
        i=i+1
        
    off_days_score=0
    for number in gap_counts:
        if number==1:
            off_days_score=off_days_score+2

    return work_days_score,off_days_score


In [34]:
def num_shifts_prev_month(shift_dates):
    num_shifts = sum(1 for element in shift_dates if element < 1)
    return num_shifts

def fewest_shifts(schedule,shift):
    fewest=100
    for doc in schedule:
        num_shifts=len(schedule[doc][shift])-num_shifts_prev_month(schedule[doc][shift])
        if num_shifts<fewest:
            return_doc=doc
            fewest=num_shifts
    return return_doc        


              
fewest_shifts(doctor_schedules,'Night')

'A'

In [40]:
doctor_schedules

{'A': {'Day1': [1, 25, 23, 14, 19, 6, 17],
  'Day2': [3, 29, 15, 24, 28, 20, 16],
  'Night': [-5, -4, -3, 21, 10, 8, 9, 7, 12]},
 'B': {'Day1': [-4, -3, -1, 24, 31, 9, 15, 20, 7, 30],
  'Day2': [-5, -2, 0, 14, 25, 21, 2, 23, 26, 13],
  'Night': [3, 4, 28, 18, 17, 11]},
 'C': {'Day1': [5, 28, 2, 3, 22, 27],
  'Day2': [-4, -3, 1, 11, 6, 4, 19, 12],
  'Night': [-2, 29, 31, 13, 20, 23, 24, 15]},
 'D': {'Day1': [-2, 16, 8, 13, 29, 21, 10],
  'Day2': [5, 18, 7, 22, 17],
  'Night': [-1, 0, 19, 26, 30, 27, 2, 25]},
 'E': {'Day1': [-5, 0, 4, 18, 26, 11, 12],
  'Day2': [-1, 27, 8, 10, 9, 31, 30],
  'Night': [16, 5, 6, 22, 1, 14]}}

In [52]:
doctor_shift_types
def check_min_shift_types(schedule):
    for doc in schedule:
        for shift in schedule[doc]:
            num_shifts=len(schedule[doc][shift])-num_shifts_prev_month(schedule[doc][shift])
            if num_shifts<doctor_shift_types[doc][shift][0] or num_shifts>doctor_shift_types[doc][shift][1] :
                return True
    return False

In [55]:
print(check_min_shift_types(doctor_schedules))

True


In [85]:
def num_d2_shifts(schedule):
    d2s=[]
    for doc in schedule:
        d2s.append(len(schedule[doc]['Day2'])-num_shifts_prev_month(schedule[doc]['Day2']))
    var=statistics.variance(d2s)
    var=var/.5
    return var


In [86]:
doctor_schedules
ex=doctor_schedules['A']['Night']
ex

[-5, -4, -3, 9, 30, 11, 19, 27, 6]

In [87]:
num_d2_shifts(doctor_schedules)

0.39999999999999997

In [99]:
import calendar
from datetime import datetime, timedelta

def get_weekends(month_year):
    # Parsing the input string to get month and year
    datetime_object = datetime.strptime(month_year, '%b %Y')
    month = datetime_object.month
    year = datetime_object.year

    # Finding the number of days in the month
    _, num_days = calendar.monthrange(year, month)

    # Creating a list of all dates in the month
    dates = [datetime(year, month, day) for day in range(1, num_days + 1)]
    
    # Filtering out the weekends (Saturday and Sunday)
    weekends = [date for date in dates if date.weekday() in (5, 6)]
    print(weekends)
    # Formatting the dates and returning as a dictionary
    return {date.strftime("%Y-%m-%d"): date.strftime("%A") for date in weekends}

# Example usage
weekends = get_weekends("Jan 2024")
print(weekends)


[datetime.datetime(2024, 1, 6, 0, 0), datetime.datetime(2024, 1, 7, 0, 0), datetime.datetime(2024, 1, 13, 0, 0), datetime.datetime(2024, 1, 14, 0, 0), datetime.datetime(2024, 1, 20, 0, 0), datetime.datetime(2024, 1, 21, 0, 0), datetime.datetime(2024, 1, 27, 0, 0), datetime.datetime(2024, 1, 28, 0, 0)]
{'2024-01-06': 'Saturday', '2024-01-07': 'Sunday', '2024-01-13': 'Saturday', '2024-01-14': 'Sunday', '2024-01-20': 'Saturday', '2024-01-21': 'Sunday', '2024-01-27': 'Saturday', '2024-01-28': 'Sunday'}


In [150]:
def get_weekends_formatted(month_year):
    # Parsing the input string to get month and year
    datetime_object = datetime.strptime(month_year, '%b %Y')
    month = datetime_object.month
    year = datetime_object.year

    # Finding the number of days in the month
    _, num_days = calendar.monthrange(year, month)

    # Creating a list of all dates in the month
    dates = [datetime(year, month, day) for day in range(1, num_days + 1)]

    # Filtering out the weekends (Saturday and Sunday)
    weekends = [date.day for date in dates if date.weekday() in (5, 6)]

    # Grouping weekends
    weekend_groups = {}
    for i, weekend_day in enumerate(weekends):
        if i % 2 == 0:
            weekend_label = {0: "first", 2: "second", 4: "third", 6: "fourth", 8: "fifth"}.get(i, "extra")
            weekend_groups[weekend_label] = [weekend_day]
        else:
            weekend_groups[weekend_label].append(weekend_day)

    return weekend_groups

# Example usage
weekends=get_weekends_formatted("Sep 2024")
print(weekends)

{'first': [1, 7], 'second': [8, 14], 'third': [15, 21], 'fourth': [22, 28], 'fifth': [29]}


In [127]:
ans

{'first': [1, 7],
 'second': [8, 14],
 'third': [15, 21],
 'fourth': [22, 28],
 'fifth': [29]}

In [170]:
# List of doctors
doctors = ["A", "B", "C", "D", "E"]

days_unavail = {"A": {"Day": [], "Night": []}, "B": {"Day": [], "Night": []}, "C": {"Day":[],"Night":[]}, "D": {"Day":[],"Night":[]}, "E": {"Day": [], "Night": []}}
# Dictionary representing the weekends
weekends = {
    'first': [1, 2],
    'second': [8, 9],
    'third': [15,16],
    'fourth': [22, 23],
    'fifth': [29]
}

# Function to assign doctors to weekends
def assign_doctors_to_weekends(doctors, weekends):
    assignment = {}
    doc_weekends_count={}
    off_shifts={}
    doctor_index = 0
    
    for each in weekends:
        off_shifts[each]=2
    for doc in doctors:
        doc_weekends_count[doc]=0
    for wkend in weekends:
        assignment[wkend]=[]
        
    for week in weekends:
        for doc in days_unavail:
            if len(set(weekends[week]).intersection(days_unavail[doc]["Day"]))>0 or len(set(weekends[week]).intersection(days_unavail[doc]["Night"]))>0:
                if off_shift[week]>0:
                    assignment[week].append(doc)
                    off_shifts[week]=off_shift[week]-1
                    doc_weekends_count[doc]=doc_weekends_count[doc]+1
        while off_shifts[week]>0:
            doc_assign='A'
            for each in doc_weekends_count:
                 if doc_weekends_count[each]==0 or doc_weekends_count[each]<doc_weekends_count[doc_assign]:
                        assignment[week].append(each)
                        doc_weekends_count[each]=doc_weekends_count[each]+1
                        off_shifts[week]=off_shifts[week]-1
                        print(off_shifts[week])
            break    
                    
    return assignment

# Creating the assignment
doctor_assignment = assign_doctors_to_weekends(doctors, weekends)
print(doctor_assignment)

1
0
-1
-2
-3
{'first': ['A', 'B', 'C', 'D', 'E'], 'second': [], 'third': [], 'fourth': [], 'fifth': []}


In [175]:
def assign_doctors_to_weekends(doctors, weekends):
    assignment = {}
    weekends_off_count = {doc: 0 for doc in doctors}
    doctor_index = 0

    for week, days in weekends.items():
        assigned_doctors = []
        for _ in range(2):  # Assign two doctors per weekend
            while True:
                current_doc = doctors[doctor_index % len(doctors)]
                doctor_index += 1

                # Check if the doctor is already unavailable or has fewer weekends off
                already_unavailable = any(day in days_unavail[current_doc]['Day'] or 
                                          day in days_unavail[current_doc]['Night'] for day in days)
                if already_unavailable or weekends_off_count[current_doc] < max(weekends_off_count.values()):
                    assigned_doctors.append(current_doc)
                    weekends_off_count[current_doc] += 1
                    break

        assignment[week] = assigned_doctors

    # Update days_unavail based on the new assignments
    for each, docs in assignment.items():
        for doc in docs:
            for day in weekends[each]:
                days_unavail[doc]["Day"].append(day)
                days_unavail[doc]["Night"].append(day)

    return assignment

# Example usage
# Assuming the 'self.days_unavail' and 'doctors' are defined elsewhere in your class
doctors = ["A", "B", "C", "D", "E"]
weekends = {'first': [1, 7], 'second': [8, 14], 'third': [15, 21], 'fourth': [22, 28], 'fifth': [29]}
assignment = assign_doctors_to_weekends(doctors, weekends)
assignment

KeyboardInterrupt: 

In [182]:
def assign_doctors_to_weekends(days_unavail, doctors, weekends):
    assignment = {}
    weekends_off_count = {doc: 0 for doc in doctors}  # Track weekends off for each doctor

    # Pre-assign doctors who are unavailable for part of a weekend
    for week, days in weekends.items():
        assignment[week] = []
        for doc in doctors:
            if any(day in days_unavail[doc]["Day"] or day in days_unavail[doc]["Night"] for day in days):
                assignment[week].append(doc)
                weekends_off_count[doc] += 1
                # Break if two doctors are already assigned to this weekend
                if len(assignment[week]) == 2:
                    break

    # Assign remaining doctors to weekends, aiming for fairness
    for week, days in weekends.items():
        if len(assignment[week]) < 2:
            sorted_doctors = sorted(doctors, key=lambda doc: weekends_off_count[doc])
            for doc in sorted_doctors:
                if doc not in assignment[week]:
                    assignment[week].append(doc)
                    weekends_off_count[doc] += 1
                    # Break if two doctors are now assigned to this weekend
                    if len(assignment[week]) == 2:
                        break

    # Update days_unavail based on the final assignment
    for week, assigned_doctors in assignment.items():
        for doc in assigned_doctors:
            for day in weekends[week]:
                if day not in days_unavail[doc]["Day"]:
                    days_unavail[doc]["Day"].append(day)
                if day not in days_unavail[doc]["Night"]:
                    days_unavail[doc]["Night"].append(day)

    return assignment

# Example usage
days_unavail = {"A": {"Day": [20, 30, 13], "Night": []}, "B": {"Day": [], "Night": []}, "C": {"Day":[],"Night":[]}, "D": {"Day":[],"Night":[]}, "E": {"Day": [], "Night": []}}
doctors = ["A", "B", "C", "D", "E"]
weekends = {'first': [6, 7], 'second': [13, 14], 'third': [20, 21], 'fourth': [27, 28], 'fifth': [30]}

assignment = assign_doctors_to_weekends(days_unavail, doctors, weekends)
print(assignment)


{'first': ['B', 'C'], 'second': ['A', 'D'], 'third': ['A', 'E'], 'fourth': ['B', 'C'], 'fifth': ['A', 'D']}


In [183]:
days_unavail

{'A': {'Day': [20, 30, 13, 14, 21], 'Night': [13, 14, 20, 21, 30]},
 'B': {'Day': [6, 7, 27, 28], 'Night': [6, 7, 27, 28]},
 'C': {'Day': [6, 7, 27, 28], 'Night': [6, 7, 27, 28]},
 'D': {'Day': [13, 14, 30], 'Night': [13, 14, 30]},
 'E': {'Day': [20, 21], 'Night': [20, 21]}}

days_unavail = {"A": {"Day": [], "Night": []}, "B": {"Day": [], "Night": []}, "C": {"Day":[],"Night":[]}, "D": {"Day":[],"Night":[]}, "E": {"Day": [], "Night": []}}
doctor_assignment

In [172]:
for each in doctor_assignment:
    for doc in doctor_assignment[each]:
        for day in weekends[each]:
            days_unavail[doc]["Day"].append(day)
            days_unavail[doc]["Night"].append(day)

print(days_unavail)        

{'A': {'Day': [1, 2], 'Night': [1, 2]}, 'B': {'Day': [1, 2], 'Night': [1, 2]}, 'C': {'Day': [1, 2], 'Night': [1, 2]}, 'D': {'Day': [1, 2], 'Night': [1, 2]}, 'E': {'Day': [1, 2], 'Night': [1, 2]}}


In [133]:
weekends

{'first': [1, 7],
 'second': [8, 14],
 'third': [15, 21],
 'fourth': [22, 28],
 'fifth': [29]}

In [148]:
import pandas as pd

def print_horizontal(doctor_schedules, num_days_in_month, weekend_days):
    # Create a DataFrame with the required structure
    df = pd.DataFrame(index=doctor_schedules.keys(), columns=range(1, num_days_in_month + 1))

    # Flatten the weekend days for easy checking
    flat_weekend_days = [day for days in weekend_days.values() for day in days]

    # Fill the DataFrame based on the schedule
    for doctor, shifts in doctor_schedules.items():
        for day in range(1, num_days_in_month + 1):
            shift_types = []
            if day in shifts.get('Day1', []):
                shift_types.append('D1')
            if day in shifts.get('Day2', []):
                shift_types.append('D2')
            if day in shifts.get('Night', []):
                shift_types.append('N')

            df.at[doctor, day] = 'DB' if len(shift_types) > 1 else ''.join(shift_types) or '·'

    # Styling function to apply colors
    def color_schedule(val):
        if val == 'D1':
            color = 'green'
        elif val == 'D2':
            color = 'blue'
        elif val == 'N':
            color = 'red'
        elif val == 'DB':
            color = 'purple'
        else:
            color = 'black'
        return f'color: {color}'

    # Define styles for weekend days in the header
    header_styles = [{
        'selector': f'th.col_heading.level0.col{col}',
        'props': [('background-color', 'lightgrey'), ('color', 'red')]  # Change color here
    } for col in flat_weekend_days]

    # Apply styles to the DataFrame
    styled_df = df.style.applymap(color_schedule)
    styled_df = styled_df.set_table_styles(header_styles)
    return styled_df


weekend_days = {
    'first': [1, 7],
    'second': [8, 14],
    'third': [15, 21],
    'fourth': [22, 28],
    'fifth': [29]
}

print_horizontal(doctor_schedules, 31, weekend_days)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
A,D2,N,·,D1,D1,D2,·,D1,D2,N,·,D1,D2,·,D2,·,D1,·,·,N,·,N,N,N,·,·,D2,N,·,D2,D1
B,·,D2,D1,N,·,·,D1,D2,N,·,·,N,·,D2,N,N,·,·,D2,·,D1,D1,D2,·,D2,D1,·,D1,D1,N,N
C,·,·,·,D2,·,D1,N,N,·,·,D2,D2,N,N,·,D1,D2,D2,·,D1,N,·,D1,D1,·,N,·,D2,·,D1,·
D,N,·,N,·,N,N,·,·,·,D2,N,·,·,D1,D1,·,·,D1,D1,D2,·,D2,·,·,D1,D2,D1,·,D2,·,D2
E,D1,D1,D2,·,D2,·,D2,·,D1,D1,D1,·,D1,·,·,D2,N,N,N,·,D2,·,·,D2,N,·,N,·,N,·,·


In [146]:
doctor_schedules

{'A': {'Day1': [12, 17, 31, 8, 5, 4],
  'Day2': [13, 6, 27, 30, 1, 15, 9],
  'Night': [-5, -4, -3, 24, 28, 2, 23, 10, 22, 20]},
 'B': {'Day1': [-4, -3, -1, 7, 22, 26, 21, 3, 28, 29],
  'Day2': [-5, -2, 0, 23, 19, 25, 2, 14, 8],
  'Night': [30, 12, 16, 4, 9, 31, 15]},
 'C': {'Day1': [20, 23, 24, 16, 30, 6],
  'Day2': [-4, -3, 18, 11, 28, 4, 12, 17],
  'Night': [-2, 7, 8, 13, 26, 14, 21]},
 'D': {'Day1': [-2, 27, 15, 14, 18, 19, 25],
  'Day2': [31, 20, 29, 22, 10, 26],
  'Night': [-1, 0, 1, 11, 6, 5, 3]},
 'E': {'Day1': [-5, 0, 13, 9, 11, 10, 2, 1],
  'Day2': [-1, 5, 24, 16, 7, 21, 3],
  'Night': [25, 29, 27, 18, 17, 19]}}